In [1]:
import pandas as pd
import numpy as np # Usado para inicializar scores mínimos

In [2]:
# --- Configurações ---
arquivo_topics = 'topics.xlsx'
arquivo_systems = 'systems.xlsx'

In [3]:
# --- Carregamento dos Dados ---
try:
    df_topics = pd.read_excel(arquivo_topics, index_col=0) # Primeira coluna (Topic) é o índice
    df_systems = pd.read_excel(arquivo_systems, index_col=0) # Primeira coluna (que chamaremos de System) é o índice
except FileNotFoundError:
    print(f"Erro: Um ou ambos os arquivos Excel ('{arquivo_topics}', '{arquivo_systems}') não foram encontrados.")
    print("Criando arquivos de exemplo...")
    # Dados de exemplo para topics.xlsx
    data_topics = {
        'Topic': ['Abstract', 'Airplane', 'Aliens', 'Alternate History', 'Assassin'],
        'Action': [2, 3, 3, 3, 3], 'Adventure': [3, -3, 1, 1, -2], 'RPG': [-3, 1, 3, 3, 3],
        'Simulation': [-3, 3, -3, 1, 1], 'Strategy': [1, 3, 2, 2, -3], 'Casual': [-3, 3, -2, -3, -3],
        'Y': [1, 3, 2, -3, -3], 'E': [2, 3, 3, 3, 1], 'M': [3, 2, 3, 3, 3]
    }
    df_topics = pd.DataFrame(data_topics).set_index('Topic')
    df_topics.to_excel(arquivo_topics)
    print(f"Arquivo '{arquivo_topics}' de exemplo criado.")

    # Dados de exemplo para systems.xlsx
    data_systems = {
        'System': ['PC', 'G64', 'TES', 'Master V', 'Gameling', 'Vena Gear'], # Renomeando coluna índice para clareza
        'Action': [2, 2, 1, 2, 1, 2], 'Adventure': [3, 3, -2, -2, -2, 1], 'RPG': [2, 2, 1, 1, 2, 1],
        'Simulation': [3, 2, 1, 1, 2, 2], 'Strategy': [3, 3, -2, -2, -3, -3], 'Casual': [-3, -2, 3, 3, 3, 3],
        'Y': [1, 1, 3, 2, 3, 2], 'E': [2, 2, -2, 3, 2, 3], 'M': [3, 3, 1, -2, -3, 1]
    }
    df_systems = pd.DataFrame(data_systems).set_index('System') # Definindo o índice correto
    df_systems.to_excel(arquivo_systems)
    print(f"Arquivo '{arquivo_systems}' de exemplo criado.")
    print("\n--- Rodando com dados de exemplo ---")

# Renomear o índice do df_systems para 'System' para clareza
df_systems = df_systems.rename_axis("System")

# --- Exibir Tabelas Originais com Gradiente (Opcional, mas útil para referência) ---
def aplicar_estilo_original(df):
    return df.style.background_gradient(cmap='RdYlGn', vmin=-3, vmax=3).format("{:.0f}")

print("\n--- Tabela de Topics (Referência) ---")
display(aplicar_estilo_original(df_topics))

print("\n--- Tabela de Systems (Referência) ---")
display(aplicar_estilo_original(df_systems))


# --- Calcular a Melhor Combinação por Atributo Específico ---

# Verificar se as colunas de atributos são as mesmas (exceto o índice)
common_attributes = df_topics.columns.intersection(df_systems.columns)
if len(common_attributes) != len(df_topics.columns) or len(common_attributes) != len(df_systems.columns):
    print("\nAviso: As colunas de atributos não são idênticas entre topics e systems.")
    print(f"Usando apenas colunas comuns: {list(common_attributes)}")
    # Filtrar DataFrames para usar apenas atributos comuns
    df_topics_filtered = df_topics[common_attributes]
    df_systems_filtered = df_systems[common_attributes]
else:
    # Usar todos os atributos se forem iguais
    df_topics_filtered = df_topics
    df_systems_filtered = df_systems
    common_attributes = df_topics.columns # Definir explicitamente para uso posterior

# DataFrames para armazenar os resultados
# df_best_system_per_attribute: Guarda o NOME do melhor sistema(s)
# df_best_score_per_attribute: Guarda o SCORE da melhor combinação
df_best_system_per_attribute = pd.DataFrame(index=df_topics_filtered.index, columns=common_attributes)
df_best_score_per_attribute = pd.DataFrame(index=df_topics_filtered.index, columns=common_attributes, dtype=float)

# Iterar por cada Topic
for topic_name, topic_scores in df_topics_filtered.iterrows():
    # Iterar por cada Atributo (Gênero/Faixa Etária)
    for attribute in common_attributes:
        topic_score = topic_scores[attribute]
        max_combined_score = -np.inf # Inicializa com um valor muito baixo
        best_systems_for_this_combo = [] # Lista para armazenar sistemas com o score máximo (caso de empate)

        # Iterar por cada System para encontrar o melhor para este Topic+Attribute
        for system_name, system_scores in df_systems_filtered.iterrows():
            system_score = system_scores[attribute]
            combined_score = topic_score + system_score

            # Verificar se encontramos um novo melhor score
            if combined_score > max_combined_score:
                max_combined_score = combined_score
                best_systems_for_this_combo = [system_name] # Começa uma nova lista com este sistema
            # Verificar se encontramos um score igual ao máximo (empate)
            elif combined_score == max_combined_score:
                best_systems_for_this_combo.append(system_name) # Adiciona à lista de melhores sistemas

        # Armazenar os resultados para este Topic+Attribute
        df_best_system_per_attribute.loc[topic_name, attribute] = ', '.join(best_systems_for_this_combo) # Junta nomes com vírgula se houver empate
        df_best_score_per_attribute.loc[topic_name, attribute] = max_combined_score


--- Tabela de Topics (Referência) ---


,Action,Adventure,RPG,Simulation,Strategy,Casual,Y,E,M
Topic,,,,,,,,,
Abstract,2,3,-3,-3,1,-3,1,2,3
Airplane,3,-3,1,3,3,3,3,3,2
Aliens,3,1,3,-3,2,-2,2,3,3
Alternate History,3,1,3,1,2,-3,-3,3,3
Assassin,3,-2,3,1,-3,-3,-3,1,3
Business,-3,1,1,3,3,-3,2,3,-2
City,-2,-3,-2,3,3,-2,2,3,1
Colonization,-2,-3,-3,3,3,-2,-2,3,1
Comedy,-3,3,1,-3,-3,3,1,2,3



--- Tabela de Systems (Referência) ---


,Action,Adventure,RPG,Simulation,Strategy,Casual,Y,E,M
System,,,,,,,,,
PC,2,3,2,3,3,-3,1,2,3
G64,2,3,2,2,3,-2,1,2,3
TES,1,-2,1,1,-2,3,3,-2,1
Master V,2,-2,1,1,-2,3,2,3,-2
Gameling,1,-2,2,2,-3,3,3,2,-3
Vena Gear,2,1,1,2,-3,3,2,3,1
Vena Oasis,3,1,1,2,-3,-2,1,3,2
Super TES,2,2,2,3,-2,2,3,2,-2
Playsystem,3,1,3,2,-2,-3,1,3,2


In [4]:
# --- Exibir a Tabela de Melhores Sistemas por Atributo ---
print("\n--- Tabela: Melhor(es) Sistema(s) por Topic e Atributo ---")
print("Cada célula mostra o(s) sistema(s) que maximiza(m) a soma (Score do Topic + Score do Sistema) para aquele atributo específico.")
display(df_best_system_per_attribute)

# --- Exibir a Tabela de Scores das Melhores Combinações com Gradiente ---
print("\n--- Tabela: Score da Melhor Combinação por Topic e Atributo ---")
print("Cada célula mostra o valor da soma máxima (Score do Topic + Score do Sistema).")
print("Cores indicam a qualidade dessa combinação (Vermelho: pior [-6], Verde: melhor [+6]).")

# Função de estilo para a tabela de scores (range -6 a 6)
def aplicar_estilo_scores_combinados(df):
    # O score combinado para um único atributo varia de -3+(-3)=-6 a 3+3=6
    return df.style.background_gradient(cmap='RdYlGn', vmin=-6, vmax=6).format("{:.0f}")

styled_scores = aplicar_estilo_scores_combinados(df_best_score_per_attribute)
display(styled_scores)

# --- Exemplo específico solicitado ---
topic_exemplo = 'Abstract'
attribute_exemplo = 'Adventure' # Usando 'Adventure' como exemplo, adapte se necessário
try:
    melhor_sistema_exemplo = df_best_system_per_attribute.loc[topic_exemplo, attribute_exemplo]
    score_exemplo = df_best_score_per_attribute.loc[topic_exemplo, attribute_exemplo]
    print(f"\nExemplo: Para Topic='{topic_exemplo}' e Atributo='{attribute_exemplo}':")
    print(f"  Melhor(es) Sistema(s): {melhor_sistema_exemplo}")
    print(f"  Score Combinado: {score_exemplo:.0f}")

    # Verificação manual (opcional)
    score_topic = df_topics_filtered.loc[topic_exemplo, attribute_exemplo]
    print(f"  (Verificação: Score Topic = {score_topic:.0f})")
    sistemas = melhor_sistema_exemplo.split(', ')
    for sistema in sistemas:
         score_system = df_systems_filtered.loc[sistema, attribute_exemplo]
         print(f"  (Verificação: Score Sistema '{sistema}' = {score_system:.0f}, Soma = {score_topic + score_system:.0f})")

except KeyError:
    print(f"\nNão foi possível encontrar o exemplo para Topic='{topic_exemplo}' e Atributo='{attribute_exemplo}'. Verifique os nomes.")


--- Tabela: Melhor(es) Sistema(s) por Topic e Atributo ---
Cada célula mostra o(s) sistema(s) que maximiza(m) a soma (Score do Topic + Score do Sistema) para aquele atributo específico.


,Action,Adventure,RPG,Simulation,Strategy,Casual,Y,E,M
Topic,,,,,,,,,
Abstract,"Vena Oasis, Playsystem, DreamVast, Playsystem ...","PC, G64","Playsystem, Playsystem 2, GS, PPS, mBox 360, P...","PC, Super TES, DreamVast, Nuu, Playsystem 3, W...","PC, G64","TES, Master V, Gameling, Vena Gear, Game Spher...","TES, Gameling, Super TES, TES 64, GS, Nuu","Master V, Vena Gear, Vena Oasis, Playsystem, D...","PC, G64, DreamVast, PPS, mBox 360"
Airplane,"Vena Oasis, Playsystem, DreamVast, Playsystem ...","PC, G64","Playsystem, Playsystem 2, GS, PPS, mBox 360, P...","PC, Super TES, DreamVast, Nuu, Playsystem 3, W...","PC, G64","TES, Master V, Gameling, Vena Gear, Game Spher...","TES, Gameling, Super TES, TES 64, GS, Nuu","Master V, Vena Gear, Vena Oasis, Playsystem, D...","PC, G64, DreamVast, PPS, mBox 360"
Aliens,"Vena Oasis, Playsystem, DreamVast, Playsystem ...","PC, G64","Playsystem, Playsystem 2, GS, PPS, mBox 360, P...","PC, Super TES, DreamVast, Nuu, Playsystem 3, W...","PC, G64","TES, Master V, Gameling, Vena Gear, Game Spher...","TES, Gameling, Super TES, TES 64, GS, Nuu","Master V, Vena Gear, Vena Oasis, Playsystem, D...","PC, G64, DreamVast, PPS, mBox 360"
Alternate History,"Vena Oasis, Playsystem, DreamVast, Playsystem ...","PC, G64","Playsystem, Playsystem 2, GS, PPS, mBox 360, P...","PC, Super TES, DreamVast, Nuu, Playsystem 3, W...","PC, G64","TES, Master V, Gameling, Vena Gear, Game Spher...","TES, Gameling, Super TES, TES 64, GS, Nuu","Master V, Vena Gear, Vena Oasis, Playsystem, D...","PC, G64, DreamVast, PPS, mBox 360"
Assassin,"Vena Oasis, Playsystem, DreamVast, Playsystem ...","PC, G64","Playsystem, Playsystem 2, GS, PPS, mBox 360, P...","PC, Super TES, DreamVast, Nuu, Playsystem 3, W...","PC, G64","TES, Master V, Gameling, Vena Gear, Game Spher...","TES, Gameling, Super TES, TES 64, GS, Nuu","Master V, Vena Gear, Vena Oasis, Playsystem, D...","PC, G64, DreamVast, PPS, mBox 360"
...,...,...,...,...,...,...,...,...,...
Virtual Pet,"Vena Oasis, Playsystem, DreamVast, Playsystem ...","PC, G64","Playsystem, Playsystem 2, GS, PPS, mBox 360, P...","PC, Super TES, DreamVast, Nuu, Playsystem 3, W...","PC, G64","TES, Master V, Gameling, Vena Gear, Game Spher...","TES, Gameling, Super TES, TES 64, GS, Nuu","Master V, Vena Gear, Vena Oasis, Playsystem, D...","PC, G64, DreamVast, PPS, mBox 360"
Vocabulary,"Vena Oasis, Playsystem, DreamVast, Playsystem ...","PC, G64","Playsystem, Playsystem 2, GS, PPS, mBox 360, P...","PC, Super TES, DreamVast, Nuu, Playsystem 3, W...","PC, G64","TES, Master V, Gameling, Vena Gear, Game Spher...","TES, Gameling, Super TES, TES 64, GS, Nuu","Master V, Vena Gear, Vena Oasis, Playsystem, D...","PC, G64, DreamVast, PPS, mBox 360"
Werewolf,"Vena Oasis, Playsystem, DreamVast, Playsystem ...","PC, G64","Playsystem, Playsystem 2, GS, PPS, mBox 360, P...","PC, Super TES, DreamVast, Nuu, Playsystem 3, W...","PC, G64","TES, Master V, Gameling, Vena Gear, Game Spher...","TES, Gameling, Super TES, TES 64, GS, Nuu","Master V, Vena Gear, Vena Oasis, Playsystem, D...","PC, G64, DreamVast, PPS, mBox 360"



--- Tabela: Score da Melhor Combinação por Topic e Atributo ---
Cada célula mostra o valor da soma máxima (Score do Topic + Score do Sistema).
Cores indicam a qualidade dessa combinação (Vermelho: pior [-6], Verde: melhor [+6]).


,Action,Adventure,RPG,Simulation,Strategy,Casual,Y,E,M
Topic,,,,,,,,,
Abstract,5,6,0,0,4,0,4,5,6
Airplane,6,0,4,6,6,6,6,6,5
Aliens,6,4,6,0,5,1,5,6,6
Alternate History,6,4,6,4,5,0,0,6,6
Assassin,6,1,6,4,0,0,0,4,6
Business,0,4,4,6,6,0,5,6,1
City,1,0,1,6,6,1,5,6,4
Colonization,1,0,0,6,6,1,1,6,4
Comedy,0,6,4,0,0,6,4,5,6



Exemplo: Para Topic='Abstract' e Atributo='Adventure':
  Melhor(es) Sistema(s): PC, G64
  Score Combinado: 6
  (Verificação: Score Topic = 3)
  (Verificação: Score Sistema 'PC' = 3, Soma = 6)
  (Verificação: Score Sistema 'G64' = 3, Soma = 6)
